In [11]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.ensemble import RandomForestRegressor
from sklearn.linear_model import LassoCV
from sklearn.model_selection import RepeatedKFold
from numpy import arange
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error
from sklearn.model_selection import GridSearchCV
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import accuracy_score
from sklearn.model_selection import RandomizedSearchCV
import warnings
warnings.filterwarnings('ignore')

In [27]:
xTrain=pd.read_csv('xTrain.csv', index_col=0)
yTrain=pd.read_csv('yTrain.csv', index_col=0)
xTest=pd.read_csv('xTest.csv', index_col=0)
yTest=pd.read_csv('yTest.csv', index_col=0)

In [28]:
xTest

,Reviews,Size,Installs,Price,Free,Paid,Adults only 18+,Everyone,Everyone 10+,Mature 17+,...,Tools,Tools;Education,Travel & Local,Travel & Local;Action & Adventure,Trivia,Video Players & Editors,Video Players & Editors;Creativity,Video Players & Editors;Music & Video,Weather,Word
6510,-0.162003,-0.740880,-0.195140,0.0,1.0,0.0,0.0,1.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
762,-0.162531,-0.128298,-0.195140,0.0,1.0,0.0,0.0,1.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2252,-0.163406,-0.084853,-0.195140,0.0,0.0,1.0,0.0,0.0,0.0,1.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2654,-0.149548,1.609524,-0.141448,0.0,1.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1945,-0.148103,0.219266,-0.086660,0.0,1.0,0.0,0.0,0.0,1.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5844,-0.072671,-0.171744,-0.086660,0.0,1.0,0.0,0.0,0.0,1.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
8003,-0.163507,-0.823427,-0.196126,0.0,0.0,1.0,0.0,1.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
515,-0.163214,-0.571443,-0.195140,0.0,1.0,0.0,0.0,0.0,0.0,1.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2345,-0.163558,1.175068,-0.196234,0.0,1.0,0.0,0.0,1.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [21]:
yTest

,numerical
6510,4.4
762,4.2
2252,4.0
2654,4.3
1945,4.7
...,...
5844,4.1
8003,3.0
515,3.5
2345,4.3


linear regression

In [31]:
reg = LinearRegression().fit(xTrain, yTrain)
yHat = reg.predict(xTest)
print(yHat)
print(yTest['numerical'])
mse_lr = mean_squared_error(yTest['numerical'], yHat)
print(mse_lr)
# print(reg.coef_)

[[4.05408478]
 [4.17236328]
 [4.19949341]
 ...
 [4.03788757]
 [4.26373291]
 [4.03715515]]
6510    4.4
762     4.2
2252    4.0
2654    4.3
1945    4.7
       ... 
5844    4.1
8003    3.0
515     3.5
2345    4.3
618     3.9
Name: numerical, Length: 2466, dtype: float64
1801846764168510.0


RandomForest Regressor

In [83]:

# Number of trees in random forest
n_estimators = [int(x) for x in np.linspace(start = 200, stop = 2000, num = 10)]
# Number of features to consider at every split
max_features = ['auto', 'sqrt']
# Maximum number of levels in tree
max_depth = [int(x) for x in np.linspace(10, 110, num = 11)]
max_depth.append(None)
# Minimum number of samples required to split a node
min_samples_split = [2, 5, 10]
# Minimum number of samples required at each leaf node
min_samples_leaf = [1, 2, 4]
# Method of selecting samples for training each tree
bootstrap = [True, False]
# Create the random grid
random_grid = {'n_estimators': n_estimators,
               'max_features': max_features,
               'max_depth': max_depth,
               'min_samples_split': min_samples_split,
               'min_samples_leaf': min_samples_leaf,
               'bootstrap': bootstrap}

In [84]:

# Use the random grid to search for best hyperparameters
# First create the base model to tune
rf = RandomForestRegressor()
# Random search of parameters, using 5 fold cross validation, 
# search across 100 different combinations, and use all available cores
rf_random = RandomizedSearchCV(estimator = rf, param_distributions = random_grid, n_iter = 50, cv = 3, verbose=2, random_state=42, n_jobs = -1)
# Fit the random search model
rf_random.fit(xTrain, yTrain)

Fitting 3 folds for each of 50 candidates, totalling 150 fits


RandomizedSearchCV(cv=3, estimator=RandomForestRegressor(), n_iter=50,
                   n_jobs=-1,
                   param_distributions={'bootstrap': [True, False],
                                        'max_depth': [10, 20, 30, 40, 50, 60,
                                                      70, 80, 90, 100, 110,
                                                      None],
                                        'max_features': ['auto', 'sqrt'],
                                        'min_samples_leaf': [1, 2, 4],
                                        'min_samples_split': [2, 5, 10],
                                        'n_estimators': [200, 400, 600, 800,
                                                         1000, 1200, 1400, 1600,
                                                         1800, 2000]},
                   random_state=42, verbose=2)

In [85]:
rf_random.best_params_

{'n_estimators': 800,
 'min_samples_split': 2,
 'min_samples_leaf': 4,
 'max_features': 'sqrt',
 'max_depth': 90,
 'bootstrap': True}

In [96]:
import math
math.sqrt(len(xTrain.columns))

2.8284271247461903

In [8]:

# Create the parameter grid based on the results of random search 
param_grid = {
    'bootstrap': [True],
    'max_depth': [90, 100],
    'max_features': [3],
    'min_samples_leaf': [3, 4, 5],
    'min_samples_split': [2, 3],
    'n_estimators': [700, 800, 900, 1000]
}
# Create a based model
rf = RandomForestRegressor()
# Instantiate the grid search model
grid_search = GridSearchCV(estimator = rf, param_grid = param_grid, 
                          cv = 3, n_jobs = -1, verbose = 2)
grid_search.fit(xTrain, yTrain)

NameError: name 'RandomForestRegressor' is not defined

In [98]:
grid_search.best_params_

{'bootstrap': True,
 'max_depth': 90,
 'max_features': 3,
 'min_samples_leaf': 5,
 'min_samples_split': 3,
 'n_estimators': 1000}

In [105]:
rf = RandomForestRegressor(bootstrap = True,
 max_depth = 90,
 max_features = 3,
 min_samples_leaf = 5,
 min_samples_split = 3,
 n_estimators =1000)
rf.fit(xTrain, yTrain['numerical'])
yHat = rf.predict(xTest)
mean_squared_error(yTest['numerical'], yHat)

0.21108135200095268

In [32]:
yTrain_kn = pd.read_csv('CateyTrain.csv', index_col=0)
yTest_kn = pd.read_csv('CateyTest.csv', index_col=0)
yTrain_kn

,1,0.5,0.2
0,4-5,4.0-4.4,4.0-4.1
1,4-5,4.5-4.9,4.6-4.7
2,4-5,4.0-4.4,4.4-4.5
3,3-4,3.5-3.9,3.6-3.7
4,4-5,4.0-4.4,4.0-4.1
...,...,...,...
5749,4-5,4.5-4.9,4.6-4.7
5750,4-5,4.0-4.4,4.0-4.1
5751,4-5,4.0-4.4,4.2-4.3
5752,4-5,4.0-4.4,4.0-4.1


In [ ]:
clf = GridSearchCV(
        KNeighborsClassifier(), 
        [{'n_neighbors': range(1,len(xTrain),100), 'metric': ['euclidean','manhattan']}], cv=5)
clf.fit(xTrain, yTrain_kn['1'])

In [127]:
clf.best_params_

{'metric': 'manhattan', 'n_neighbors': 1}

In [14]:
KN = KNeighborsClassifier(metric = 'manhattan', n_neighbors = 1)
KN.fit(xTrain, yTrain_kn['1'])
yHat = KN.predict(xTest)
print(yHat)

['4-5' '4-5' '3-4' ... '3-4' '3-4' '4-5']


In [15]:
accuracy_score(yHat, yTest_kn['1'])

0.6994301994301995

In [16]:
clf = GridSearchCV(
        KNeighborsClassifier(), 
        [{'n_neighbors': range(1,len(xTrain),100), 'metric': ['euclidean','manhattan']}], cv=5)
clf.fit(xTrain, yTrain_kn['0.5'])
clf.best_params_

{'metric': 'manhattan', 'n_neighbors': 301}

In [17]:
KN = KNeighborsClassifier(metric = 'manhattan', n_neighbors = 301)
KN.fit(xTrain, yTrain_kn['0.5'])
yHat = KN.predict(xTest)
print(yHat)
accuracy_score(yHat, yTest_kn['0.5'])

['4.0-4.4' '4.0-4.4' '4.0-4.4' ... '4.0-4.4' '4.0-4.4' '4.0-4.4']


0.48824786324786323

In [18]:
clf = GridSearchCV(
        KNeighborsClassifier(), 
        [{'n_neighbors': range(1,len(xTrain),100), 'metric': ['euclidean','manhattan']}], cv=5)
clf.fit(xTrain, yTrain_kn['0.2'])
clf.best_params_

{'metric': 'euclidean', 'n_neighbors': 701}

In [19]:
KN = KNeighborsClassifier(metric = 'manhattan', n_neighbors = 701)
KN.fit(xTrain, yTrain_kn['0.2'])
yHat = KN.predict(xTest)
print(yHat)
accuracy_score(yHat, yTest_kn['0.2'])

['4.4-4.5' '4.4-4.5' '4.4-4.5' ... '4.4-4.5' '4.4-4.5' '4.4-4.5']


0.3108974358974359